# Train and visualize a model in Tensorflow - Part 1 (optional): Dataset Preprocessing

This notebook is optional for those who want to follow how we got the dataset we are using for the tutorial. For this tutorial you need to previously do all the configuration steps explained in **tutorial 0**, plus install the optional libraries.

## Dataset Download

The task we choose to work in this tutorial is document classification using the 20 Newsgroup Corpus, which is a standard resource for such task. It is a corpus of emails with a topic. For information on the 20 Newsgroup Corpus please refer to the [official website of the project](http://qwone.com/~jason/20Newsgroups/). 

For this tutorial we will be using the dataset with duplicates removed and only "From" and "Subject" headers. It is the file named [20news-18828.tar.gz](http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz) (`md5: ed94957ee54863606ef4e13d84025230`) (if there is any problem you can try with our [mirror](https://cs.famaf.unc.edu.ar/~ccardellino/pydata/20news-18828.tar.gz)).

For the word embeddings we will be using Word2Vec's pre-trained embeddings on the [Google News corpus](https://cs.famaf.unc.edu.ar/~ccardellino/pydata/GoogleNews-vectors-negative300.bin.gz) (`md5: 1c892c4707a8a1a508b01a01735c0339`).

### Extracting the data

I will assume the 20 Newsgroup corpus was extracted to a directory `./resources/20news-18828/`. In this directory, following the original downloaded file structure, there are many subdirectories with the name of the category. Each subdirectory has many documents inside of the category (one document per file).

The Word2Vec's model should be inside the directory `./resources/` and it doesn't need to be unzipped as gensim can read it as it is.

In [1]:
from __future__ import absolute_import, print_function, unicode_literals

import numpy as np
import os

from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

## Helper function to traverse directory

As we need to go through all the 20 newsgroup files the following function is similar to UNIX's `find` and yields the full path of all the files given a directory.

In [2]:
def find_files(path):
    for root, _, filenames in os.walk(path):
        for filename in filenames:
            yield os.path.join(root, filename)

## Documents labels

The document classification task takes a representation of a document and feeds it to a classifier to obtain the category (or topic) of such document. The 20 newsgroup has (as the name indicates) 20 different categories. The labels of such categories are represented by the name of the subdirectories as explained before. So, to gather those categories we get each file and extract the name of the directory containing it. We sort the files so we can match them after to the matrix representation of the documents.

We convert the string labels into integer labels so we can use them in TensorFlow.

In [3]:
newsgroup_files = sorted(find_files('./resources/20news-18828/'))

# `labels` holds the name of the labels and `target` holds the representation of the labels using integers
labels, target = np.unique([os.path.basename(os.path.dirname(fname)) for fname in newsgroup_files],
                           return_inverse=True)

## Document representation via Tf-Idf

For document classification we need a way to represent the documents as a numerical matrix. A commong technique to do so is by using a [bag-of-words model](https://en.wikipedia.org/wiki/Bag-of-words_model) of the documents. This technique consists in counting the number of occurring word in each document. Then it represents the document as positive integer vector where each dimension is a word in the vocabulary and shows how many times that word appeared in the document.

As natural language follows a [Zipfian distribution](https://en.wikipedia.org/wiki/Zipf%27s_law) some word occur a lot
of time. Generally those words have less significance in representing a document since they appear in many documents. Thus, a way of weighting the significance of the word is with [Term Frequency-Inverse Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (Tf-Idf). The idea is precisely to weight the word by the number of times it appear and inversely by the number of documents it appear.

To do it, Scikit Learn has the [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) which given a set of documents turns them into a tf-idf matrix. In this case we decided to remove the English stopwords (words with high frequency count that doesn't add to the meaning of a document, like connectors, articles, etc.)

In [4]:
vectorizer = TfidfVectorizer(input='filename', decode_error='replace', stop_words='english')
document_matrix = vectorizer.fit_transform(newsgroup_files)

## Document representation via word embeddings

Now we have the document matrix (which is a tf-idf matrix). We could just use that as our input matrix. This, however, is not the only way. In part because the number of features can be too high for a tensorflow model to handle it. Another reason is because word embeddings are what all the cool people is using now. But mostly because they generally go well hand in hand with neural networks.

To do so we first need to load Word2Vec's model from the Google News corpus. We use `gensim` to do it. What we load has some properties similar to a hash table (or dictionary).

In [5]:
model = KeyedVectors.load_word2vec_format('./resources/GoogleNews-vectors-negative300.bin.gz', binary=True)

With this model we create a matrix with the embedding on each word in the document matrix obtained before. To simplify, if the word is not present in the model we try a lower case version of the word and if still not present we just leave that word as zero.

In [6]:
# The `embedding_matrix` has one row for each word in the `document_matrix` and
# a number of columns equal to the number of dimensions of the Word2Vec's model
embedding_matrix = np.zeros((document_matrix.shape[1], model.vector_size))

# We traverse all the vocabulary and set the embedding for such word in 
# the vocabulary (in the correct order)
for word, idx in vectorizer.vocabulary_.items():
    if word in model:
        embedding_matrix[idx, :] = model[word]
    elif word.lower() in model:
        embedding_matrix[idx, :] = model[word.lower()]

### How to represent a document by its word vectors?

There are different ways to represent a document by the word vectors it has. Examples can be the sum or average of the vectors of the words conforming the document. There is some [work](https://arxiv.org/abs/1607.00570) done using pointwise minimum and maximum operations to represent a short sentence.

After some preliminary experiments we decided the simpler way to do so, that showed very good results whatsoever, was using a sum weighted by the tf-idf score of each word. To obtain so was straight from the matrices we already have: the tf-idf document matrix and the embeddings matrix. We only need to multiply those matrices to obtain our final document-embedding matrix.

In [7]:
document_embedding_matrix = document_matrix.dot(embedding_matrix)

## Splitting dataset in train/test subsets and saving the file

Finally, once we have both the document-embedding matrix and the labels (both in string format and the representation with integers) we can split the dataset into train and test. To do so we use [stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling) which is already implemented in Scikit Learn.

Finally we save the datasets using numpy's compress format.

In [8]:
sss = StratifiedShuffleSplit(2, test_size=0.2, random_state=0)
train_idx, test_idx = next(sss.split(document_embedding_matrix, target))

train_data = document_embedding_matrix[train_idx]
train_target = target[train_idx]

test_data = document_embedding_matrix[test_idx]
test_target = target[test_idx]

np.savez_compressed('./resources/newsgroup.npz',
                    train_data=train_data, train_target=train_target,
                    test_data=test_data, test_target=test_target,
                    labels=labels)